In [1]:
using Mmap
using Match
using BenchmarkTools
using Profile
using Distributed
using TestImages, Images, ImageView
using Profile
using Dates
Threads.nthreads()


Gtk-Message: 17:51:32.409: Failed to load module "xapp-gtk3-module"
Gtk-Message: 17:51:32.409: Failed to load module "canberra-gtk-module"


8

In [2]:
function readtoend()
    res = UInt64(0)
    f = open("./COX3.txt", "r")
    line = ""
    while !eof(f)
        line = readline(f)
#         res += sum(line)
    end
    line
end

function mmap_readtoend(step::Int)
    res = UInt64(0)
    f = open("./COX3.txt", "r")
    m = Mmap.mmap(f)
    block = zeros(UInt8, step)
    s = 1
    e = 1
    l = length(m)
    while e < l
        e += step
        if e > l
            e = l
        end
        
        res += sum(m[s: e])
        s += step
        
    end
    res
end

# @time readtoend()
# @time mmap_readtoend(65536)

mmap_readtoend (generic function with 1 method)

In [42]:
function get_mmap()
    f = open("/home/myuan/projects/data/T311-Gm42418.txt", "r")
#     f = open("/home/myuan/projects/data/T33-all.txt", "r")
#     f = open("./T33-head.txt", "r")
    m = Mmap.mmap(f)
    m
end

function addx(x::UInt, n::UInt8)
    return (x * 10 + n)::UInt
end

function mmap_read_limit(start_pos::Int, end_pos::Int)

    m = get_mmap()
    current_pos = start_pos
    current_status = 1

    l = length(m)
    if end_pos === 0
        end_pos = l
    end

    current_x = UInt(0)
    current_y = UInt(0)
    max_x = max_y = UInt(0)
    min_x = min_y = typemax(UInt)

    while current_pos < end_pos && current_pos < l
        @inbounds current_char = m[current_pos]
        if current_char === UInt8('\t')
            current_status += 1 
        elseif current_char === UInt8('\n')
            current_status = 1
            max_x = max(max_x, current_x)
            max_y = max(max_y, current_y)
            min_x = min(min_x, current_x)
            min_y = min(min_y, current_y)
            current_x = current_y = UInt(0)
#             println("$max_x, $max_y, $min_x, $min_y\n")
        else
            if current_status === 1 
                current_x = addx(current_x, current_char - UInt8('0'))::UInt
            elseif current_status === 2
                current_y = addx(current_y, current_char - UInt8('0'))::UInt
            end

        end


        current_pos += 1
    #     print("$current_pos, $current_status, $current_x, $current_y, $(Char(current_char))\n")
    #     if current_pos > 30
    #         break
    #     end
    end
#     print("$current_pos, $current_status, $current_x, $current_y, $(Char(current_char))\n")
    current_pos, max_x, max_y, min_x, min_y
end

mmap_read_limit (generic function with 1 method)

In [4]:
# @benchmark mmap_read_limit(1, 0)

In [5]:
# @time mmap_read_limit(1, 0)

In [6]:
# @code_warntype mmap_read_limit(1, 0)

## 并行版本

In [7]:
function find_line_end_helper(m::Vector{UInt8}, start::Int)
    
    while start <= length(m)
        if m[start] == UInt8('\n')
            break
        else
            start += 1
        end
    end
    
    return start
end


function find_line_end(m::Vector{UInt8}, chunk_size::Int)
    l = length(m)
    positions = Tuple{Int, Int}[]
    start = 1
    
    single_size = floor(Int, l / chunk_size)
    
    for i = 1:chunk_size
        new_start = start + single_size
        if new_start > l
            new_start = l
        end
        
        new_start = find_line_end_helper(m, new_start)
        
        push!(positions, (start, new_start))
        start = new_start + 1
    end
    positions
end
    

find_line_end (generic function with 1 method)

In [8]:
# @time find_line_end(get_mmap(), 1)
# @code_warntype find_line_end(get_mmap(), 1)

In [9]:
function mmap_read_limit_pmap(size::Int)
    m = get_mmap()
    line_ends = find_line_end(m, size)
    pmap(i -> mmap_read_limit(i[1], i[2]), line_ends)
end
# @time mmap_read_limit_pmap(6)

mmap_read_limit_pmap (generic function with 1 method)

In [10]:
function mmap_read_limit_thr(size::Int)
    m = get_mmap()
    line_ends = find_line_end(m, size)
    max_x = max_y = UInt(0)
    min_x = min_y = typemax(UInt)

    Threads.@threads for i = line_ends
        _, maxx, maxy, minx, miny = mmap_read_limit(i[1], i[2])
        
        max_x = max(max_x, maxx)
        max_y = max(max_y, maxy)
        min_x = min(min_x, minx)
        min_y = min(min_y, miny)

    end
    max_x, max_y, min_x, min_y
end



mmap_read_limit_thr (generic function with 1 method)

In [11]:
# @time mmap_read_limit_thr(8)


In [12]:
bin_size = 50
@time max_x, max_y, min_x, min_y = mmap_read_limit_thr(8)


  0.313062 seconds (120.60 k allocations: 7.665 MiB, 8.06% compilation time)


(0x000000000000ef3f, 0x000000000001d5ba, 0x0000000000000000, 0x0000000000003977)

In [44]:
function get_bin_value(v::UInt, bin_size::Int, min_v::UInt)::UInt
    ceil(UInt, (v - min_v) / bin_size) 
end

function get_zero_img(bin_size, max_y, min_y, max_x, min_x)
    zeros(UInt16, get_bin_value(max_y, bin_size, min_y) + 1, get_bin_value(max_x, bin_size, min_x) + 1)
end

function mmap_get_bin_res_limit_res(start_pos::Int, end_pos::Int, max_x::UInt, max_y::UInt, min_x::UInt, min_y::UInt, bin_size::Int)
    m = get_mmap()
    current_pos = start_pos
    current_status = 1

    l = length(m)
    if end_pos === 0
        end_pos = l
    end

    current_x = UInt(0)
    current_y = UInt(0)
    current_count = UInt(0)
    matrix = get_zero_img(bin_size, max_y, min_y, max_x, min_x)

    while current_pos < end_pos && current_pos < l
        current_char = m[current_pos]
#         println("current_char: $current_char")
        if current_char === UInt8('\t')
            current_status += 1 
        elseif current_char === UInt8('\r')
            
        elseif current_char === UInt8('\n')
            current_status = 1
            new_y = get_bin_value(current_y + 1, bin_size, min_y) 
            new_x = get_bin_value(current_x + 1, bin_size, min_x) 
#             println("$current_y, $current_x, $new_y, $new_x, $current_count\n")
            
#             try
                matrix[new_y, new_x] += current_count
#             catch e
#                 println(e)
#                 println("$current_y, $current_x, $new_y, $new_x, $current_count, l: $l, current_pos: $current_pos, end_pos: $end_pos\n")
                 
#             end
            
            current_count = current_x = current_y = UInt(0)
#             println("$max_x, $max_y, $min_x, $min_y\n")
        else
            if current_status === 1 
                current_x = addx(current_x, current_char - UInt8('0'))::UInt
            elseif current_status === 2
                current_y = addx(current_y, current_char - UInt8('0'))::UInt
            elseif current_status === 3
#                 println("$current_count, $(current_char - UInt8('0'))")
                current_count = addx(current_count, current_char - UInt8('0'))::UInt
            end

        end
        current_pos += 1
    end
    matrix
end

@time t = mmap_get_bin_res_limit_res(1, 405206303, max_x, max_y, min_x, min_y, bin_size)
# Profile.print(format=:flat)
sum(t)

  0.297950 seconds (19 allocations: 4.940 MiB, 6.60% gc time)


0x000000000085b27a

In [30]:
length(get_mmap())

405206303

In [40]:
function mmap_get_bin_res_limit_thr(size::Int)
    img = get_zero_img(bin_size, max_y, min_y, max_x, min_x)
    m = get_mmap()
    line_ends = find_line_end(m, size)
    add_lock = Threads.Atomic{Int}(0)
    println("start")
    
    Threads.@threads for i = 1:size
#         mmap_get_bin_res_limit(view(imgs, i, :, :), line_ends[i][1], line_ends[i][2], max_x, max_y, min_x, min_y, bin_size)
        res = mmap_get_bin_res_limit_res(line_ends[i][1], line_ends[i][2], max_x, max_y, min_x, min_y, bin_size)
#         waiting_time = 0.001
#         while add_lock.value != 0
#             println("$i is wating, current: $(add_lock.value)")
#             sleep(waiting_time)
#             waiting_time += rand() / 1000
#         end
#         Threads.atomic_add!(add_lock, 1)
#         println("$i got lock")
#         img += res
#         Threads.atomic_add!(add_lock, -1)
    end
    
#     for i = 1:size
#         img += imgs[i, :, :]
#     end
    img
#     sum(imgs, dims=1)[1, :, :]
end

@benchmark t = mmap_get_bin_res_limit_thr(8)
# Profile.print()
# sum(t)

start
start
start
start
start
start
start
start
start
start
start
start
start
start
start
start
start
start
start
start
start
start


BenchmarkTools.Trial: 10 samples with 1 evaluation.
 Range (min … max):  438.996 ms … 644.021 ms  ┊ GC (min … max): 0.00% … 0.83%
 Time  (median):     500.108 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   517.937 ms ±  56.646 ms  ┊ GC (mean ± σ):  0.10% ± 0.26%

  ▁              ▁▁▁█▁    ▁                  ▁                ▁  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁█████▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  439 ms           Histogram: frequency by time          644 ms <

 Memory estimate: 44.47 MiB, allocs estimate: 338.